In [1]:
import numpy as np
from time import sleep
from neurons import poisson, LIF
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import mpl
output_notebook()

Loading BokehJS ...

In [8]:
class spiking_neural_network:
    def __init__(self):
        self.x = []
        self.y = []
        self.layer0 = []
        self.layer1 = []
        self.layer2 = []
        self.layer0_eval = []
        self.w01 = []
        self.w12 = []
    
    def generate_network(self):
        for i in range(2):
            self.layer0.append(poisson())
        for i in range(15):
            self.layer1.append(LIF())
        for i in range(1):
            self.layer2.append(LIF())
            self.layer2[i].update_threshold(4)

        self.w01 = np.random.rand(2, 15)
        self.w12 = np.random.rand(15, 1)

    def test(self, x, time):
        dt = time[1] - time[0]
        
        for i in range(len(self.layer0)):
            self.layer0_eval.append(self.layer0[i].execute(x[i], time))

        syn01 = []
        for i in range(len(self.layer1)):
            sum = 0
            for j in range(len(self.layer0)):
                sum += self.layer0_eval[j]*self.w01[j, i]
            syn01.append(sum)
        syn01 = np.array(syn01)

        for i, t in enumerate(time):
            for j in range(len(self.layer1)):
                self.layer1[j].execute(syn01[j, i], t)

            for j in range(len(self.layer2)):
                syn = 0
                for k in range(len(self.layer1)):
                    syn += self.w12[k, j]*self.layer1[k].out
                self.layer2[j].execute(syn, t)
            
            # sleep(dt/100)
        

In [47]:
T = 100
dt = 0.5
time = np.arange(0, T+dt, dt)

x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

snn = spiking_neural_network()
snn.generate_network()
snn.test([0.5, 0.8], time)

p = figure(height=400)
color = mpl['Plasma'][3]
for i in range(len(snn.layer2)):
    # p.vbar(x=time, top=snn.layer2[i].out_array, color=color[i], legend_label='LIF {}'.format(i+1))
    p.line(x=time, y=snn.layer2[i].pot_plot, color=color[i], legend_label='LIF {}'.format(i+1), line_width=1.5)
show(p)   